In [248]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import re
import pandas as pd

In [57]:
quote_page = 'https://www.zillow.com/homes/for_sale/Boston-MA-02126/59138273_zpid/58640_rid/42.300388,-71.060172,42.251615,-71.119309_rect/13_zm/'

In [58]:
req = urllib.request.Request(
    quote_page, 
    data=None, 
    headers={
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    })

In [59]:
page = urlopen(req)

In [60]:
page_response = requests.get(quote_page, timeout=5)

In [61]:
soup = BeautifulSoup(page, 'html.parser')

In [109]:
name.prettify()

'<header class="zsg-content-header addr">\n <h1 class="notranslate">\n  19 Ellison Ave,\n  <span class="zsg-h2 addr_city">\n   Boston, MA 02126\n  </span>\n </h1>\n <h3 class="">\n  <span aria-hidden="true" class="middle-dot">\n  </span>\n  <span class="addr_bbs">\n   5 beds\n  </span>\n  <span aria-hidden="true" class="middle-dot">\n  </span>\n  <span class="addr_bbs">\n   3 baths\n  </span>\n  <span aria-hidden="true" class="middle-dot">\n  </span>\n  <span class="addr_bbs">\n   1,772 sqft\n  </span>\n </h3>\n</header>'

In [134]:
name.find(class_='notranslate').get_text()

'19 Ellison Ave, Boston, MA 02126 '

In [152]:
def get_location(soup):
    # Retrieve location information
    name = soup.find(class_='zsg-content-header')
    address = name.find(class_='notranslate').get_text()
    address, city, state_zipcode = address.split(',')

    # Strip whitespace on the right and left side
    city = city.lstrip().rstrip()
    state, zipcode = state_zipcode.lstrip().rstrip().split(' ')
    
    return address, city, state, zipcode

In [294]:
name = soup.find(class_='zsg-content-section')
facts = name.find_all(class_='hdp-fact-ataglance-value')

header = ['house_type', 'year_built', 'heating', 'cooling', 'parking', 'lot_size', 'days_on_zillow', 'price_per_sqft', 'saves',
         'bedroom', 'full_bath', 'half_bath', 'floor_size']
all_facts = [x.get_text() for x in facts]
all_facts[5] = int(re.findall('\d+', all_facts[5].replace(',',''))[0])
all_facts[7] = int(re.findall('\d+', all_facts[7].replace(',',''))[0])
name = soup.find(class_='hdp-fact-container-columns')
facts = name.find_all(class_='hdp-fact-value')
bed, bath, size = [x.get_text() for x in facts]
bed = int(bed)
full_bath, half_bath = list(map(int, re.findall('\d', bath)))
size = int(re.findall('\d+', size.replace(',',''))[0])

all_facts.extend([bed, full_bath, half_bath, size])
all_facts = pd.DataFrame(all_facts, index=header).T

In [295]:
all_facts

,house_type,year_built,heating,cooling,parking,lot_size,days_on_zillow,price_per_sqft,saves,bedroom,full_bath,half_bath,floor_size
0,Single Family,1925,No Data,No Data,No Data,5227,Less than 1 day,395,No Data,5,2,1,1772
